In [13]:
from pathlib import Path
from pprint import pprint

import torch 
import yaml
from pytorch_metric_learning.utils.accuracy_calculator import AccuracyCalculator, precision_at_k
from omegaconf.dictconfig import DictConfig
from torch.utils.data import DataLoader

from retrieval.engine.evaluate import evaluate, get_tester
from retrieval.models.net import RetrievalNet
from retrieval.getter import Getter

DEVICE = "cuda"
CKPT_PATH = Path("./checkpoints/deit_roadmap.ckpt")
getter = Getter()

In [14]:
CKPT = torch.load(CKPT_PATH)

In [15]:
CKPT.keys()

dict_keys(['net_state', 'optimizer_state', 'scheduler_on_epoch_state', 'scheduler_on_step_state', 'scheduler_on_val_state', 'scaler_state', 'epoch', 'seed', 'config', 'score', 'best_score', 'best_model', 'RANDOM_STATE', 'NP_STATE', 'TORCH_STATE', 'TORCH_CUDA_STATE'])

In [16]:
pprint(CKPT['config'])

{'experience': {'experiment_name': 'GLDv2_ROADMAP_classification_splits_v2_batch${dataset.sampler.kwargs.batch_size}', 'log_dir': '/home/and/projects/made/landmarks-recognition/andrey-research-repo/experiments/ROADMAP', 'seed': 42, 'resume': '/home/and/projects/made/landmarks-recognition/andrey-research-repo/experiments/ROADMAP/GLDv2_ROADMAP_classification_splits128/weights/rolling.ckpt', 'maybe_resume': False, 'force_lr': None, 'warm_up': -1, 'warm_up_key': 'fc', 'max_iter': 1300, 'train_eval_freq': -1, 'val_eval_freq': -1, 'test_eval_freq': 5, 'eval_bs': 96, 'save_model': 20, 'eval_split': 'test', 'principal_metric': 'mean_average_precision_at_r_level0', 'log_grad': False, 'with_AP': False, 'landmarks': False, 'split': None, 'kfold': None, 'split_random_state': None, 'with_super_labels': False, 'num_workers': 4, 'pin_memory': True, 'sub_batch': 128, 'update_type': 'base_update'}, 'dataset': {'name': 'GLDv2DatasetClassificationSplits', 'kwargs': {'data_dir': '${env:HOME}/projects/made

In [17]:
net = RetrievalNet(
    "vit_deit_distilled",
    embed_dim=384,
    norm_features=False,
    without_fc=True,
    with_autocast=False,
)
weights = torch.load(CKPT_PATH, map_location=DEVICE)["net_state"]
net.load_state_dict(weights)
net.to(DEVICE)
net.eval()

RetrievalNet(
  (backbone): VisionTransformer(
    (patch_embed): PatchEmbed(
      (proj): Conv2d(3, 384, kernel_size=(16, 16), stride=(16, 16))
      (norm): Identity()
    )
    (pos_drop): Dropout(p=0.0, inplace=False)
    (norm_pre): Identity()
    (blocks): Sequential(
      (0): Block(
        (norm1): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (attn): Attention(
          (qkv): Linear(in_features=384, out_features=1152, bias=True)
          (attn_drop): Dropout(p=0.0, inplace=False)
          (proj): Linear(in_features=384, out_features=384, bias=True)
          (proj_drop): Dropout(p=0.0, inplace=False)
        )
        (ls1): Identity()
        (drop_path1): Identity()
        (norm2): LayerNorm((384,), eps=1e-06, elementwise_affine=True)
        (mlp): Mlp(
          (fc1): Linear(in_features=384, out_features=1536, bias=True)
          (act): GELU(approximate=none)
          (drop1): Dropout(p=0.0, inplace=False)
          (fc2): Linear(in_features=1536